# Sentiment Analysis Bag-of-Words

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

## 1. Data Loading

In [4]:
import random
data = pd.read_csv ("train.tsv", sep = '\t')

# Teilsätze rausschmeißen
data = data.groupby('SentenceId').first().reset_index()
data.head()


# so Daten laden, wenn das test_set Sentiment Labels hätte:
#train_set = pd.read_csv ("train.tsv", sep = '\t')
#test_set= pd.read_csv ("test.tsv", sep = '\t')
#pd.set_option('display.max_colwidth', None)
#test_set.head(10)

,SentenceId,PhraseId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,64,"This quiet , introspective and entertaining in...",4
2,3,82,"Even fans of Ismail Merchant 's work , I suspe...",1
3,4,117,A positively thrilling combination of ethnogra...,3
4,5,157,Aggressive self-glorification and a manipulati...,1


In [5]:
data.Sentiment.value_counts()

# Daten ausbalancieren im Trainings-Set? 
# Jeder Sentiment Value sollte gleiche Anzahl an Samples haben 

3    2321
1    2200
2    1655
4    1281
0    1072
Name: Sentiment, dtype: int64

## 2. Data cleaning

In [6]:
import nltk
# nltk.download('stopwords')
# pip install nltk
import re


# Für Sentimentanalyse zählen nur Wörter  
def keep_only_letters(text):
    text=re.sub(r'[^a-zA-Z\s]','',text)
    return text
 
# Groß- und Kleinschreibung egal 
def convert_to_lowercase(text):
    return text.lower()
 
def clean_reviews(text):
    text = keep_only_letters(text)
    text = convert_to_lowercase(text)
    return text

data['Phrase'] = data['Phrase'].apply(clean_reviews)
# train_set['Phrase'] = train_set['Phrase'].apply(clean_reviews)
# test_set['Phrase'] = test_set['Phrase'].apply(clean_reviews)

# Stop Words definition
english_stop_words = nltk.corpus.stopwords.words('english')
print(len(english_stop_words))
print (english_stop_words[:20])

# Stop Words removal
def remove_stop_words(text):
    for stopword in english_stop_words:
        stopword = ' ' + stopword + ' '
        text = text.replace(stopword, ' ')
    return text
 

data['Phrase'] = data['Phrase'].apply(remove_stop_words) 
#train_set['Phrase'] = train_set['Phrase'].apply(remove_stop_words)
#test_set['Phrase'] = test_set['Phrase'].apply(remove_stop_words)


# Stemming
def text_stemming(text):
    stemmer = nltk.porter.PorterStemmer()
    stemmed = ' '.join([stemmer.stem(token) for token in text.split()])
    return stemmed

data['Phrase'] = data['Phrase'].apply(text_stemming) 
#train_set['Phrase'] = train_set['Phrase'].apply(text_stemming)
#test_set['Phrase'] = test_set['Phrase'].apply(text_stemming)

#train_set.head(10)
#train_set.shape
#test_set.shape

179
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his']


In [7]:
# Train Test Split
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(data, test_size=0.2)

## 3. Text Vectorization
CountVectorizer aus sklearn um bag-of-words Darstellung von unserem Trainings- und Testset zu erhalten
naive bag-of-words text vectorization

Nur Trainingsdatensatz zur Definition des Vokabulars heranziehen und 
das gleiche Vokabular zur Darstellung des Test-Datensatzes verwenden
-> Vektorizer an Trainingsdaten anpassen und zur Transformation der Testdaten verwenden

weighted version of BOW ausprobieren?

### N-Grams

Unigramme: Alle eindeutigen Wörter in einem Dokument

BiGramme: Alle Permutationen von zwei aufeinanderfolgenden Wörtern in einem Dokument

TriGrams: Alle Permutationen von drei aufeinanderfolgenden Wörtern in einem Dokument

In [8]:
import sklearn

# Argument binary=False: Vocabel-Vector mit term-frequency füllen
# binary = True: Vocabel-Vector mit Vorhandensein der Token füllen (1 vorhanden, 0 nicht vorhanden) 
# ngram _range = Unigram: (1,1); Bigram: (1,2); Trigram: (1,3)

vectorizer_uni = sklearn.feature_extraction.text.CountVectorizer(binary=False,ngram_range=(1,1))
vectorizer_bi = sklearn.feature_extraction.text.CountVectorizer(binary=False,ngram_range=(1,2))
vectorizer_tri = sklearn.feature_extraction.text.CountVectorizer(binary=False,ngram_range=(1,3))

uni_features_train = vectorizer_uni.fit_transform(train_set['Phrase'])
uni_features_test = vectorizer_uni.transform(test_set['Phrase'])
#print (uni_features_train.shape, uni_features_test.shape)

bi_features_train = vectorizer_bi.fit_transform(train_set['Phrase'])
bi_features_test = vectorizer_bi.transform(test_set['Phrase'])
#print (bi_features_train.shape, bi_features_test.shape)

tri_features_train = vectorizer_tri.fit_transform(train_set['Phrase'])
tri_features_test = vectorizer_tri.transform(test_set['Phrase'])
#print (tri_features_train.shape, tri_features_test.shape)


Unigram:
(156060, 10998) (66292, 10998): 10998 einzigartige englische token in unserem Vokabular (abgeleitet aus Trainingsdatensatz)
Jeder Token wird durch eine Spalte im Datensatz repräsentiert
Für jedes Review im Datensatz wird die Frequency der Token (term-frequency) durch Vokabel-Vector der Größe 10998 dargestellt.
= Daher haben wir 156060 solcher Vektoren in unserem Trainings-Datensatz und 66292 in unserem Test-Datensatz = Anzahl der Reviews 

In [9]:
train_labels = train_set['Sentiment']
test_labels = test_set['Sentiment']

## 4. Klassifizierungsmodelle trainieren
### 4.1.  Naive Bayes
#### 4.1.1. Unigram (Logistic Regression classifier on unigram features)

In [10]:
from sklearn.naive_bayes import MultinomialNB
 
vectorizer_uni = sklearn.feature_extraction.text.CountVectorizer(binary=False,ngram_range=(1,1))
uni_features_train = vectorizer_uni.fit_transform(train_set['Phrase'])
uni_features_test = vectorizer_uni.transform(test_set['Phrase'])

uni_nb = MultinomialNB()
uni_nb.fit(uni_features_train, train_labels)
 
predictions = uni_nb.predict(uni_features_test)
print(sklearn.metrics.classification_report(test_labels, predictions, target_names=['0','1','2','3','4']))
print(sklearn.metrics.confusion_matrix(test_labels, predictions, labels=[0, 1, 2, 3, 4]))


              precision    recall  f1-score   support

           0       0.42      0.11      0.17       238
           1       0.38      0.57      0.46       418
           2       0.31      0.16      0.21       334
           3       0.40      0.59      0.48       485
           4       0.41      0.25      0.31       231

    accuracy                           0.39      1706
   macro avg       0.38      0.34      0.33      1706
weighted avg       0.38      0.39      0.36      1706

[[ 25 143  25  43   2]
 [ 22 240  34 109  13]
 [  8 121  53 134  18]
 [  5  96  49 285  50]
 [  0  31   9 133  58]]


#### 4.1.2 Naive Bayes: Unigram + Bigram

In [13]:
#vectorizer_bi = sklearn.feature_extraction.text.CountVectorizer(binary=False,ngram_range=(1,2))
#bi_features_train = vectorizer_bi.fit_transform(train_set['Phrase'])
#bi_features_test = vectorizer_bi.transform(test_set['Phrase'])

bi_nb = MultinomialNB()
bi_nb.fit(bi_features_train, train_labels)
 
predictions = bi_nb.predict(bi_features_test)
print(sklearn.metrics.classification_report(test_labels, predictions, target_names=['0','1','2','3','4']))
print(sklearn.metrics.confusion_matrix(test_labels, predictions, labels=[0, 1, 2, 3, 4]))


              precision    recall  f1-score   support

           0       0.50      0.07      0.12       238
           1       0.38      0.63      0.47       418
           2       0.31      0.10      0.15       334
           3       0.40      0.67      0.50       485
           4       0.45      0.13      0.20       231

    accuracy                           0.39      1706
   macro avg       0.41      0.32      0.29      1706
weighted avg       0.40      0.39      0.33      1706

[[ 16 157  19  44   2]
 [ 11 263  21 118   5]
 [  3 133  32 160   6]
 [  2 108  26 325  24]
 [  0  30   5 166  30]]


#### 4.1.3 Naive Bayes: Unigram + Bigram + Trigram

In [12]:
tri_nb = MultinomialNB()
tri_nb.fit(tri_features_train, train_labels)
 
predictions = tri_nb.predict(tri_features_test)
print(sklearn.metrics.classification_report(test_labels, predictions, target_names=['0','1','2','3','4']))
print(sklearn.metrics.confusion_matrix(test_labels, predictions, labels=[0, 1, 2, 3, 4]))


              precision    recall  f1-score   support

           0       0.50      0.05      0.10       238
           1       0.38      0.64      0.48       418
           2       0.34      0.09      0.14       334
           3       0.40      0.69      0.51       485
           4       0.46      0.11      0.18       231

    accuracy                           0.39      1706
   macro avg       0.42      0.32      0.28      1706
weighted avg       0.41      0.39      0.33      1706

[[ 13 159  17  47   2]
 [  9 269  15 120   5]
 [  3 136  29 161   5]
 [  1 112  19 336  17]
 [  0  31   5 170  25]]


### 4.2 Logistische Regression
#### 4.2.1. Unigram

In [15]:
uni_logreg = sklearn.linear_model.LogisticRegression(max_iter=10000)
uni_logreg.fit(uni_features_train, train_labels)
#print (uni_logreg)

""" default state of the classifier: 
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)"""

# Predictions für unseren Test-Datensatz, Accuracy, Confusion-Matrix berechnen: 

predictions = uni_logreg.predict(uni_features_test)
print(sklearn.metrics.classification_report(test_labels, predictions, target_names=['0','1','2','3','4']))
print(sklearn.metrics.confusion_matrix(test_labels, predictions, labels=[0, 1, 2, 3, 4]))


              precision    recall  f1-score   support

           0       0.33      0.18      0.23       238
           1       0.36      0.43      0.39       418
           2       0.29      0.28      0.28       334
           3       0.42      0.47      0.44       485
           4       0.38      0.37      0.38       231

    accuracy                           0.37      1706
   macro avg       0.36      0.34      0.35      1706
weighted avg       0.36      0.37      0.36      1706

[[ 43 115  46  29   5]
 [ 58 180  77  81  22]
 [ 12 108  92  98  24]
 [ 15  83  76 226  85]
 [  4  18  22 102  85]]


#### 4.2.2. Log Regression: Unigrams + Bigrams

In [21]:
bi_logreg = sklearn.linear_model.LogisticRegression(max_iter=10000)
bi_logreg.fit(bi_features_train, train_labels)
#print (bi_logreg)

predictions = bi_logreg.predict(bi_features_test)
print(sklearn.metrics.classification_report(test_labels, predictions, target_names=['0','1','2','3','4']))
print(sklearn.metrics.confusion_matrix(test_labels, predictions, labels=[0, 1, 2, 3, 4]))
# Feature-Set Größe erhöht sich weil auch Bi-Gramme berücksichtigt 

              precision    recall  f1-score   support

           0       0.39      0.13      0.20       238
           1       0.38      0.50      0.43       418
           2       0.28      0.24      0.26       334
           3       0.41      0.50      0.45       485
           4       0.41      0.32      0.36       231

    accuracy                           0.37      1706
   macro avg       0.37      0.34      0.34      1706
weighted avg       0.37      0.37      0.36      1706

[[ 32 119  43  38   6]
 [ 35 210  72  83  18]
 [  9 112  80 115  18]
 [  4  97  77 241  66]
 [  3  21  17 116  74]]


#### 4.2.3. Log Regression: Unigrams + Bigrams + Trigrams

In [18]:
tri_logreg = sklearn.linear_model.LogisticRegression(max_iter=10000)
tri_logreg.fit(tri_features_train, train_labels)
#print (tri_logreg)

predictions = tri_logreg.predict(tri_features_test)
print(sklearn.metrics.classification_report(test_labels, predictions, target_names=['0','1','2','3','4']))
print(sklearn.metrics.confusion_matrix(test_labels, predictions, labels=[0, 1, 2, 3, 4]))

              precision    recall  f1-score   support

           0       0.43      0.12      0.19       238
           1       0.38      0.52      0.44       418
           2       0.27      0.22      0.25       334
           3       0.40      0.50      0.44       485
           4       0.37      0.27      0.32       231

    accuracy                           0.37      1706
   macro avg       0.37      0.33      0.33      1706
weighted avg       0.37      0.37      0.35      1706

[[ 29 126  41  36   6]
 [ 23 219  70  90  16]
 [  8 115  75 119  17]
 [  5 100  71 242  67]
 [  2  23  19 124  63]]


### 4.3 Linear Support Vector Machine (LSVM) 
#### 4.3.1. Unigram 

In [60]:
uni_lsvm = sklearn.svm.LinearSVC()
uni_lsvm.fit(uni_features_train, train_labels)
 
predictions = uni_lsvm.predict(uni_features_test)
print(sklearn.metrics.classification_report(test_labels, predictions, target_names=['0','1','2','3','4']))
print(sklearn.metrics.confusion_matrix(test_labels, predictions, labels=[0, 1, 2, 3, 4]))

              precision    recall  f1-score   support

           0       0.27      0.24      0.25       225
           1       0.31      0.34      0.32       427
           2       0.27      0.24      0.25       339
           3       0.38      0.44      0.40       427
           4       0.47      0.41      0.44       288

    accuracy                           0.34      1706
   macro avg       0.34      0.33      0.33      1706
weighted avg       0.34      0.34      0.34      1706

[[ 53 105  33  28   6]
 [ 79 145  84  95  24]
 [ 29 116  80  90  24]
 [ 29  64  72 186  76]
 [ 10  37  28  96 117]]


#### 4.3.2. LSVM: Unigram + Bigram 

In [19]:
bi_lsvm = sklearn.svm.LinearSVC()
bi_lsvm.fit(bi_features_train, train_labels)
 
predictions = bi_lsvm.predict(bi_features_test)
print(sklearn.metrics.classification_report(test_labels, predictions, target_names=['0','1','2','3','4']))
print(sklearn.metrics.confusion_matrix(test_labels, predictions, labels=[0, 1, 2, 3, 4]))

              precision    recall  f1-score   support

           0       0.38      0.21      0.27       238
           1       0.35      0.40      0.38       418
           2       0.28      0.28      0.28       334
           3       0.43      0.46      0.44       485
           4       0.36      0.39      0.37       231

    accuracy                           0.37      1706
   macro avg       0.36      0.35      0.35      1706
weighted avg       0.36      0.37      0.36      1706

[[ 51 109  42  30   6]
 [ 56 169  90  70  33]
 [ 14  97  93  98  32]
 [ 13  81  84 221  86]
 [  2  22  20  98  89]]
